In [ ]:
!wget https://data.vision.ee.ethz.ch/ihnatova/camera_scene_detection_validation.zip

In [3]:
import shutil
shutil.unpack_archive("/content/mobilenet_v2_mai.tar.gz", "/content/")

In [6]:
!wget https://data.vision.ee.ethz.ch/ihnatova/camera_scene_detection_validation.zip

--2021-03-01 07:48:06--  https://data.vision.ee.ethz.ch/ihnatova/camera_scene_detection_validation.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20050014 (19M) [application/zip]
Saving to: ‘camera_scene_detection_validation.zip’

camera_scene_detect 100%[===================>]  19.12M  15.6MB/s    in 1.2s    

2021-03-01 07:48:08 (15.6 MB/s) - ‘camera_scene_detection_validation.zip’ saved [20050014/20050014]



In [7]:
!unzip /content/camera_scene_detection_validation.zip

Archive:  /content/camera_scene_detection_validation.zip
   creating: images/
  inflating: images/0.jpg            
  inflating: images/1.jpg            
  inflating: images/10.jpg           
  inflating: images/100.jpg          
  inflating: images/101.jpg          
  inflating: images/102.jpg          
  inflating: images/103.jpg          
  inflating: images/104.jpg          
  inflating: images/105.jpg          
  inflating: images/106.jpg          
  inflating: images/107.jpg          
  inflating: images/108.jpg          
  inflating: images/109.jpg          
  inflating: images/11.jpg           
  inflating: images/110.jpg          
  inflating: images/111.jpg          
  inflating: images/112.jpg          
  inflating: images/113.jpg          
  inflating: images/114.jpg          
  inflating: images/115.jpg          
  inflating: images/116.jpg          
  inflating: images/117.jpg          
  inflating: images/118.jpg          
  inflating: images/119.jpg          
  inflatin

In [4]:
TF_LITE_FILE = 'mobilenetv2_int.tflite'

In [10]:
import tensorflow as tf

In [12]:
import os
print(len(os.listdir('images/')))

600


In [29]:
import cv2

images = []
size = 224,224

for image in os.listdir('images/'):
    temp_img = cv2.imread('images/' + image)
    temp_img = cv2.resize(temp_img, size)
    images.append(temp_img)

In [32]:
import numpy as np
images = np.array(images)

In [33]:
print(images.shape)
print(images.dtype)

(600, 224, 224, 3)
uint8


In [11]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_FILE)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [  1 224 224   3]
Input Type: <class 'numpy.uint8'>
Output Shape: [ 1 30]
Output Type: <class 'numpy.uint8'>


In [15]:
interpreter.resize_tensor_input(input_details[0]['index'], (600, 224, 224, 3))
interpreter.resize_tensor_input(output_details[0]['index'], (600, 30))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [600 224 224   3]
Input Type: <class 'numpy.uint8'>
Output Shape: [600  30]
Output Type: <class 'numpy.uint8'>


In [34]:
interpreter.set_tensor(input_details[0]['index'], images)
interpreter.invoke()

tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])

print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

Prediction results shape: (600, 30)


In [35]:
prediction_classes

array([15, 29, 15,  8, 22,  5, 25, 23,  6,  1,  7,  5,  4, 21, 20,  3,  1,
        3, 16, 24,  3,  2, 19, 27, 27, 15,  0, 28, 23, 11, 19,  7, 17,  0,
       20, 12,  6, 23, 11, 23, 17, 20, 10, 14, 26, 20,  1, 17, 25,  9, 18,
        8, 13, 13,  1, 22, 17,  1,  3, 21,  5, 29, 17, 21,  5, 15, 24, 24,
       27, 12,  9,  1,  9, 17, 16, 11, 27, 19, 13,  5, 19, 12, 26, 23, 26,
       22, 16, 25, 14, 29,  3, 22, 29,  7, 10, 19,  8, 17, 28,  2, 28,  4,
       27, 23, 29,  2,  0, 20,  9,  4, 17, 16,  9,  6, 24,  3, 29, 25, 18,
       24, 15,  3,  7, 17, 13, 25, 18, 13,  1,  3, 22, 13, 18, 10, 16, 17,
       26, 11,  6,  4, 16,  5, 13, 13, 22, 15, 27,  0, 14,  7, 16, 25, 10,
        3, 10, 23, 15, 10, 12, 23,  3, 14, 21,  8, 15, 26, 12, 26, 20, 28,
        1, 11,  3, 19, 26,  4, 15, 21, 23, 26,  4, 13, 26, 21, 18, 28, 25,
       22,  6, 11,  5, 12, 19, 20, 24, 13, 26, 24, 15, 24, 13,  4,  9, 21,
        3,  3, 17, 20, 27, 26, 12,  4, 12,  4, 18,  6,  0, 26, 18,  2, 19,
       25,  6, 18, 25,  2